In [2]:
import pandas as pd
from collections import Counter
import numpy as np

# df = pd.read_csv("fake_data.csv", encoding_errors="ignore", on_bad_lines='skip', sep=",",)
df = pd.read_csv("BPI2016_Clicks_Logged_In.csv", encoding_errors="ignore", on_bad_lines='skip', sep=";",
                usecols=['CustomerID', 'AgeCategory', 'Gender', 'Office_U', 'Office_W',
       'SessionID', 'IPID', 'TIMESTAMP', 'VHOST', 'URL_FILE', 'PAGE_NAME',
       'REF_URL_category', 'page_load_error', 'page_action_detail', 'tip',
       'service_detail', 'xps_info'])
df.rename(columns={'URL_FILE':'Activity'}, inplace=True)

C:\Users\krdeg\AppData\Local\Temp\ipykernel_10620\3933864525.py:6: DtypeWarning: Columns (11,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("BPI2016_Clicks_Logged_In.csv", encoding_errors="ignore", on_bad_lines='skip', sep=";",


In [3]:
df["TIMESTAMP"] = pd.to_datetime(df["TIMESTAMP"], infer_datetime_format=True)
df = df.sort_values(["SessionID", "TIMESTAMP"])

In [4]:
#Create consecutive column with the consecutive activity and count how many time each pair occurs
df["Consecutive_1"] = df.groupby("SessionID")["Activity"].shift(periods=-1)

#Count all the pairs and safe in new DataFrame
df_for_pivot_1 = pd.DataFrame(Counter(list(zip(df['Activity'], df['Consecutive_1'],))), index=["value"]).T.reset_index()

df_for_pivot_1.fillna("end_session", inplace=True)
df_for_pivot_1

,level_0,level_1,value
0,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,123143
1,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...,2442
2,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,21692
3,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,241943
4,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,end_session,53821
...,...,...,...
15942,/xpsimage/wdo215419,/xpsimage/wdo215419,3
15943,/xpsimage/wdo213133,/xpsimage/wdo215419,1
15944,/xpsimage/wdo215419,/xpsimage/wdo211812,1
15945,/portal/page/portal/werk_nl/werknemer/sollicit...,/werk_nl/werknemer/solliciteren/solliciteren-w...,1


In [5]:
def make_pivot(df, index_names, column_names):
    """
    Functions that takes in a DataFrame and returns a pivot table with all the chances
    
    Create pivot table where chances are calculated that each row is succeded by activity that is represented in the column
    """
    df_chances = df.pivot_table(index=index_names, columns=column_names, values='value')
    
    #Calculate what the probabilities are by summing the row and dividing all the values in the row by total sum of the row
    df_chances["total_row_count"] = df_chances.sum(axis=1)
    df_chances = df_chances.div(df_chances["total_row_count"], axis=0)
    
    df_chances.fillna(0, inplace=True)
    df_chances.drop("total_row_count", axis=1, inplace=True)

    return df_chances

In [6]:
df_chances_1 = make_pivot(df_for_pivot_1, "level_0", "level_1")

#Calculate the probability of each activity that it is the first activity performed in the session
df_chances_1["start_session_chance"] = df.groupby("SessionID").nth(0)["Activity"].value_counts() / df.groupby("SessionID").nth(0)["Activity"].value_counts().sum()
df_chances_1.fillna(0, inplace=True)

In [7]:
df_chances_total = df_chances_1 

#Split the start probabilities from the normal DataFrame
start_chances = df_chances_total["start_session_chance"][df_chances_total["start_session_chance"] > 0]

#Create df with all chances except the starting chance.
df_chances_rest = df_chances_total.iloc[:, :-1]

In [8]:
df_chances_rest

level_1,/,//,//portal/page/portal/werk_nl/werknemer/solliciteren/solliciteren-werk-zoeken/sollicitatiebrief,//werk_nl/werknemer/contact,//werk_nl/werknemer/contact/email_uw_vraag,//werk_nl/werknemer/contact/vestiging-zoeken,//werk_nl/werknemer/home,//werk_nl/werknemer/over-werk-nl/handleiding,//werk_nl/werknemer/over-werk-nl/handleiding/cv-plaatsen,//werk_nl/werknemer/over-werk-nl/handleiding/sollicitatie-wijziging,...,/xpsitem/wdo_013389,/xpsitem/wdo_013407,/xpsitem/wdo_013827,/xpsitem/wdo_014041,/zoeken_portlet/ajax/addVacatureToGereageerd,/zoeken_portlet/ajax/zoekAantalIndicatief,/zoeken_portlet/ajax/zoekBeroep,/zoeken_portlet_wg/ajax/selectContactpersoon,/zoeken_portlet_wg/ajax/selectVacature,end_session
level_0,,,,,,,,,,,,,,,,,,,,,
/,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.073834
//,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.035714
//portal/page/portal/werk_nl/werknemer/solliciteren/solliciteren-werk-zoeken/sollicitatiebrief,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
//werk_nl/werknemer/contact,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
//werk_nl/werknemer/contact/email_uw_vraag,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.058824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/zoeken_portlet/ajax/addVacatureToGereageerd,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017588,0.002513,0.005025,0.0,0.0,0.055276
/zoeken_portlet/ajax/zoekAantalIndicatief,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000107,0.537184,0.051269,0.0,0.0,0.007539
/zoeken_portlet/ajax/zoekBeroep,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.441592,0.489910,0.0,0.0,0.003979


Try to find paths of length 3.
A - B - C
Sort on chance. 
So chance a to b * b to c 

In [33]:
def create_test_df_circle():
  d = {'A': [0,0,0,0],'B': [1,0.2,0,0.2], 'C': [0,0.4,0,0],'D':[0,0.4,1,0],'E':[0,0,0,0.8]}
  df = pd.DataFrame(data=d)
  df.index = ['A','B','C','D']
  return df

def create_test_df():
  d = {'A': [0,0,0,0],'B': [1,0,0,0.2], 'C': [0,0.6,0,0],'D':[0,0.4,1,0],'E':[0,0,0,0.8]}
  df = pd.DataFrame(data=d)
  df.index = ['A','B','C','D']
  return df

def create_test_df_simple():
  d = {'A': [0,0,0,0],'B': [1,0,0,0], 'C': [0,1,0,0],'D':[0,0,1,0],'E':[0,0,0,1]}
  df = pd.DataFrame(data=d)
  df.index = ['A','B','C','D']
  return df

test_df = create_test_df_simple() 
# test_df = create_test_df_circle() 
test_df 

,A,B,C,D,E
A,0,1,0,0,0
B,0,0,1,0,0
C,0,0,0,1,0
D,0,0,0,0,1


In [38]:
def swapping(_df:pd.DataFrame,event_1:str,event_2:str,deviation_rate):
  df = _df.copy()
  deviation_rate = deviation_rate / 100
  print(deviation_rate)
  print(f"swapping event {event_1} with {event_2} with deviation rate: {deviation_rate}")
  
  for i,val in df.loc[:,event_1].items():
    if i != event_2 and val > 0:
      old_val = df.loc[i,event_1]
      df.loc[i,event_1] *= 1 - deviation_rate
      if i != event_1:
        df.loc[i,event_2] += old_val - df.loc[i,event_1]
  
  df.loc[event_2,event_1] +=  df.loc[event_1,event_2] * deviation_rate
  df.loc[event_2, :] = df.loc[event_2, :] / df.loc[event_2, :].sum()
      
  return df

res = swapping(test_df,'B','C',10)
res
sum_check = res.copy()
if not "Sum" in sum_check: sum_check['Sum'] = sum_check.sum(axis=1)
sum_check

0.1
swapping event B with C with deviation rate: 0.1


,A,B,C,D,E,Sum
A,0,0.9,0.1,0,0,1.0
B,0,0.0,1.0,0,0,1.0
C,0,0.1,0.0,1,0,1.1
D,0,0.0,0.0,0,1,1.0


In [11]:
# # sum_check = new_df.copy()
# if not "Sum" in sum_check: sum_check['Sum'] = sum_check.sum(axis=1)


# if (sum_check['Sum'] == sum_check['Sum'][0]).all():
#     print("All values are equal in column 'Sum'")
# else:
#     print("All values are not equal  in column 'Sum'")

# sum_check.sort_values(['Sum'])
# # sum_check['Sum'].to_csv("test.csv")

In [12]:
def transform_df_to_dict(df):
    df_dict = df.T.to_dict()
    for i in df_dict:
        df_dict[i] = {k: v for k, v in df_dict[i].items() if v > 0}
        
    return df_dict

df_dict = transform_df_to_dict(new_df)

NameError: name 'new_df' is not defined

In [ ]:
def markov_chain(amount, start_chances, df_dict):
    lijst_aantal_gegenereerde = []

    #Loop through the df with all the chances and based on this a new session is generated
    aanvragen_ww_list = []
    while len(aanvragen_ww_list) <= amount:
        activity_list = []
        activity = np.random.choice(a = np.array(start_chances.index), size = 1, p = np.array(start_chances.values))
        activity_list.append(activity[0])

        while activity != "end_session":
            row_activity = df_dict[activity[0]]
            activity = np.random.choice(a = list(row_activity.keys()), size = 1, p = list(row_activity.values()))
            activity_list.append(activity[0])

        if "/portal/page/portal/home/diensten/aanvragen-ww" in activity_list:
            aanvragen_ww_list.append(activity_list)

        #print how many sessions are already generated by the Markov chain. Only print every 250 generated sessions
        if len(aanvragen_ww_list) % 250 == 0 and len(aanvragen_ww_list) not in lijst_aantal_gegenereerde:
            print(len(aanvragen_ww_list))
            lijst_aantal_gegenereerde.append(len(aanvragen_ww_list))
    
    return aanvragen_ww_list

markov_result = markov_chain(2000, start_chances, df_dict)
# markov_result

0
250
500
750
1000
1250
1500
1750
2000
